In [1]:
import csv

In [2]:
with open('influencers.csv') as f:
    reader = csv.reader(f)
    next(reader)
    data = []
    for row in (reader):
        data.append((int(row[0]),int(row[1]),float(row[2])))

In [3]:
def follower_fuzzification(dataset):
    fuzzy = []
    many = 0.0
    nor = 0.0
    few = 0.0
    for i in range(len(dataset)):
        if(dataset[i][1]<=15000):
            few = 1.0
        if((dataset[i][1]>15000) and (dataset[i][1]<=20000)):
            few = (20000-dataset[i][1])/(20000-15000)
        if(dataset[i][1]>20000):
            few = 0.0 
        if((dataset[i][1]>15000) and (dataset[i][1]<=25000)):
            nor = (dataset[i][1]-15000)/(25000-15000)
        if((dataset[i][1]>25000 and dataset[i][1]<=40000)):
            nor = 1.0
        if((dataset[i][1]>40000) and (dataset[i][1]<=50000)):
            nor = (50000-dataset[i][1])/(50000-40000)
        if(dataset[i][1]>50000 or (dataset[i][1]<=15000)):
            nor = 0.0
        if(dataset[i][1]<=33000):
            many = 0.0
        if((dataset[i][1]>33000) and (dataset[i][1]<=43000)):
            many = (dataset[i][1]-33000)/(43000-33000)
        if(dataset[i][1]>43000):
            many = 1.0
        fuzzy.append([dataset[i][0],few,nor,many])
    return fuzzy

In [4]:
def engagement_fuzzification(dataset):
    fuzzy = []
    high = 0.0
    mid = 0.0
    low = 0.0
    for i in range(len(dataset)):
        if(dataset[i][2]<=0.8):
            low = 1.0
        if((dataset[i][2]>0.8) and (dataset[i][2]<=1)):
            low = (1-dataset[i][2])/(1-0.8)
        if(dataset[i][2]>1):
            low = 0.0 
        if((dataset[i][2]>0.8) and (dataset[i][2]<=1.3)):
            mid = (dataset[i][2]-0.8)/(1.3-0.8)
        if((dataset[i][2]>1.3 and dataset[i][2]<=2.2)):
            mid = 1.0
        if((dataset[i][2]>2.2) and (dataset[i][2]<=3.5)):
            mid = (3.5-dataset[i][2])/(3.5-2.2)
        if((dataset[i][2]>3.5) and (dataset[i][2]<=0.8)):
            mid = 0.0
        if(dataset[i][2]<=2.5):
            high = 0.0
        if((dataset[i][2]>2.5) and (dataset[i][2]<=3.2)):
            high = (dataset[i][2]-2.5)/(3.2-2.5)
        if(dataset[i][2]>3.2):
            high = 1.0
        fuzzy.append([dataset[i][0],low,mid,high])
    return fuzzy

In [5]:
fol = follower_fuzzification(data)
eng = engagement_fuzzification(data)

In [6]:
def accept_inference_min(fol_lin,eng_lin):
    rules = []
    for i in range(len(data)):
        rules.append([fol_lin[i][0],min(fol_lin[i][3],eng_lin[i][3]),min(fol_lin[i][3],eng_lin[i][2]),min(fol_lin[i][2],eng_lin[i][3])])
    return rules

In [7]:
def considered_inference_min(fol_lin,eng_lin):
    rules = []
    for i in range(len(data)):
        rules.append([fol_lin[i][0],min(fol_lin[i][3],eng_lin[i][1]),min(fol_lin[i][2],eng_lin[i][2]),min(fol_lin[i][1],eng_lin[i][3])])
    return rules

In [8]:
def rejected_inference_min(fol_lin,eng_lin):
    rules = []
    for i in range(len(data)):
        rules.append([fol_lin[i][0],min(fol_lin[i][2],eng_lin[i][1]),min(fol_lin[i][1],eng_lin[i][2]),min(fol_lin[i][1],eng_lin[i][1])])
    return rules

In [9]:
acc = accept_inference_min(fol,eng)
con = considered_inference_min(fol,eng)
rej = rejected_inference_min(fol,eng)

In [10]:
def accept_inference_max(accepted):
    inference = []
    for i in range(len(data)):
        inference.append([accepted[i][0],max(accepted[i][1],accepted[i][2],accepted[i][3])])
    return inference

In [11]:
def considered_inference_max(considered):
    inference = []
    for i in range(len(data)):
        inference.append([considered[i][0],max(considered[i][1],considered[i][2],considered[i][3])])
    return inference

In [12]:
def rejected_inference_max(rejected):
    inference = []
    for i in range(len(data)):
        inference.append([rejected[i][0],max(rejected[i][1],rejected[i][2],rejected[i][3])])
    return inference

In [13]:
acc_inf = accept_inference_max(acc)
con_inf = considered_inference_max(con)
rej_inf = rejected_inference_max(rej)

In [25]:
def defuzzification(accepted_inf,considered_inf,rejected_inf):
    crisp_output = []
    for i in range(len(data)):
        crisp_output.append([((accepted_inf[i][1]*100)+(considered_inf[i][1]*70)+(rejected_inf[i][1]*50))/(accepted_inf[i][1]+considered_inf[i][1]+rejected_inf[i][1]),accepted_inf[i][0]])
    return crisp_output

In [29]:
result = defuzzification(acc_inf,con_inf,rej_inf)
result.sort(reverse = True)
selected = []
for i in range(20):
    selected.append(result[i][1])
selected

[72, 61, 60, 59, 53, 41, 25, 19, 13, 11, 75, 67, 63, 76, 36, 6, 91, 2, 80, 48]

In [30]:
with open('chosen.csv', mode='w') as chosen:
    chosen_writer = csv.writer(chosen)
    chosen_writer.writerow(selected)